## Imports

In [1]:
import sys
sys.path.append('python/')
import numpy as np
import lmfit as lf
import matplotlib.pyplot as plt
import scipy.optimize as opt
import dataPython as dp
from scipy.integrate import quad
import scipy.optimize as so
import scipy.special as ss

In [2]:
## Measured points
#Measured points from Noordermeer's paper:

In [3]:
#data = dp.getXYdata_wXYerr('data/NGC5533-rot-data_fmt.txt')
#r_dat = np.asarray(data['xx'])
#v_dat = np.asarray(data['yy'])
#v_err0 = np.asarray(data['ex'])
#v_err1 = np.asarray(data['ey'])

## Dark Halo

In [4]:
##Noordermeer's Dark Halo curve
data_dm_nord = dp.getXYdata('data/NGC5533-dm_fmt.txt')

#convert to numpy arrays
r_dm_nord = np.asarray(data_dm_nord['xx'])
v_dm_nord = np.asarray(data_dm_nord['yy'])

In [23]:
##Constants (Initial Guesses)
gamma = 0
h = 8.9                                                     #radial scale-length (kpc)
rc = 1.4                                                    #core radius (kpc)
G = 4.300e-6                                                #gravitational constant (kpc/solar mass*(km/s)^2)
rho00 = 0.31e9                                              #central surface density (solar mass/kpc^3)

##Definitions
#isothermal density profile
rho = lambda r,rho00,rc: rho00*((1+((r/rc)**2))**(-1))      

#function from isothermal density profile
f = lambda R,rho00,rc: 4*np.pi*rho(R,rho00,rc)*(R**2)      

#mass equation
m = lambda r,rho00,rc: quad(f, 0, r, args=(r,rho00,rc,))[0]                           
   
##Main Function
def v(r,rho00,rc,G):
    return np.sqrt((G*m(r,rho00,rc))/r)

In [24]:
#SCIPY FITTING
#Setup
#Initial Guesses for rho00,rc,G. In this case, our initial guess is the true function.
p0 = [0.31e9,1.4,4.3e-6]                  #Parameters should be in the same order here as they were when we defined our function.
bounds = [0,np.inf]                       #KITTYADD bc sqrt can't be negative

#Do fit
s_fit = opt.curve_fit(v,r_dm_nord,v_dm_nord,p0,bounds=bounds,absolute_sigma=True) #absolute_sigma is set so that uncertainties aren't treated as percentages.
print(s_fit)                              #If we uncomment this line, we see that s_fit is an array containing two arrays.

#Define parameters from fit. Our parameters are stored in the first array in our fit output, in the order they were listed in f.
s_rho00 = s_fit[0][0]
s_rc = s_fit[0][1]
s_G = s_fit[0][2]

#Define error from fit.
s_cov = s_fit[1]                          #The second array in the fit output is a covariance matrix.
s_error = np.sqrt(np.diag(s_cov))         #The diagonals of the covariance matrix are the variances of individual parameters.
s_rho00e = s_error[0]                     #The errors will be in an order corresponding to the order of the parameters in their array
s_rce = s_error[1]
s_Ge = s_error[2]


#isothermal density profile
s_rho = lambda r,s_rho00,s_rc: s_rho00*((1+((r/s_rc)**2))**(-1))      

#function from isothermal density profile
s_f = lambda R,s_rho00,s_rc: 4*np.pi*s_rho(R,s_rho00,s_rc)*(s_R**2)      

#mass equation
s_m = lambda r,s_rho00,s_rc: quad(f, 0, r, args=(r,s_rho00,s_rc,))[0]   


#Create array to plot
s_curve = np.sqrt((s_Ge*m(r,s_rho00e,s_rce))/r_dm_nord)

#Print Values
print('rho00: '+str(s_rho00e)+u' \u00B1 '+str(s_rho00e))
print('rc: '+str(s_rce)+u' \u00B1 '+str(s_rce))
print('G: '+str(s_G)+u' \u00B1 '+str(s_Ge))

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [16]:
#LMFIT FITTING
#Setup
#weighdata = 1/sigdata                      #We will need weights for lmfit. This tells us how much to account for a single data point in the fit.
l_mod = lf.Model(v)                         #Tell lmfit that we want to model the function f
params = l_mod.make_params(rho00=0.31e9,rc=1.4,G=4.3e-6)   #Give lmfit our initial guesses - again, the true function
params.add('rho00', value=0.31e9, min=0) 
params.add('rc', value=1.4, min=0)          #so lmfit doesn't guess negative values 
params.add('G', value=4.3e-6, min=0)        #so lmfit doesn't guess negative values (bc G is so close to zero)

#Do fit
l_fit = l_mod.fit(v_dm_nord, params, r=r_dm_nord, nan_policy='omit') #Here is where the weights we set at the beginning come in.

#Define Stuff
l_dict = l_fit.best_values                  #l_fit has a lot of output. We want to use the final result.
l_rho00 = l_dict['rho00']                   #Dictionary items are called based on their name.
l_rc = l_dict['rc']                         #So, we don't have to track the order of parameters.
l_G = l_dict['G']

#Create array to plot
l_curve = np.sqrt((l_G*m(r_dm_nord,l_rho00,l_rc))/r_dm_nord)        #just equation only now with these second array things

l_fit                                       #Display information about the fit

ValueError: The truth value of an array with more than one element is ambiguous. Use a.any() or a.all()

In [ ]:
#Plotting

fig = plt.figure(figsize=(9.0,8.0))                #size of the plot

plt.errorbar(r_dm_nord,v_dm_nord,fmt='bo',label='Data') #Plot points (fmt='*o') with error bars
#plt.plot(r_dm_nord,ycurve,label='Function')                      #Accepting default style gives us a solid line
plt.plot(r_dm_nord,s_curve,linestyle='--',label='SciPy')         #Plot the scipy curve fit with a dashed line
plt.plot(r_dm_nord,l_curve,linestyle='--',label='LmFit')

plt.legend()           #Tells our plot to show a legend
plt.show()             #Depending how your environment is set up, this line may not be necessary, but it won't break anything.